In [6]:
import pandas as pd
import pickle
import sys
from utils import load_data
from scipy.sparse import coo_matrix
import seaborn as sns
import numpy as np
from tqdm import tqdm

In [2]:
browsing, search, sku, info = load_data('../dataset/new/')

load browsing done...
load search done...
load sku done...
load info done...


In [3]:
search

session_id_hash  \
train 0              1498632   
      1              1498641   
      2              1498647   
      3              1498647   
      4              1498647   
...                      ...   
test  16914          5076719   
      16915          5076747   
      16916          5076749   
      16917          5076750   
      16918          5076752   

                                                  query_vector  \
train 0      [-0.20255649089813232, -0.016908567398786545, ...   
      1      [-0.007610442116856575, -0.14909175038337708, ...   
      2      [-0.20023074746131897, -0.03151938319206238, 0...   
      3      [-0.18556387722492218, -0.07620412111282349, 0...   
      4      [-0.03269264101982117, -0.27234694361686707, 0...   
...                                                        ...   
test  16914  [-0.1711975336074829, 0.008784637786448002, 0....   
      16915  [-0.21009723842144012, -0.37940073013305664, 0...   
      16916  [0.09223505854606628, 0.20877793431282043, 0.0...   
      16917  [-0.0683397725224495, -0.1326630413532257, -0....   
      16918  [0.059666723012924194, -0.09347045421600342, -...   

            clicked_skus_hash  \
train 0                    []   
      1                    []   
      2                    []   
      3                    []   
      4                    []   
...                       ...   
test  16914                []   
      16915           [32004]   
      16916                []   
      16917           [17899]   
      16918      [27746, 499]   

                                             product_skus_hash  \
train 0                                                     []   
      1                                  [20133, 44952, 46326]   
      2       [24916, 65372, 1640, 55027, 40991, 52685, 36305]   
      3      [65035, 37475, 3766, 52978, 65850, 33711, 1248...   
      4      [22249, 44982, 16330, 32905, 7367, 12983, 7233...   
...                                                        ...   
test  16914                 [35348, 20424, 18802, 5155, 50488]   
      16915  [24812, 8816, 7237, 17386, 3711, 65389, 125, 1...   
      16916                                                 []   
      16917           [1605, 9626, 17899, 54837, 50245, 16800]   
      16918                   [13492, 45882, 499, 7109, 27746]   

             server_timestamp_epoch_ms  
train 0                  1548575194779  
      1                  1548276763869  
      2                  1548937997295  
      3                  1548938038268  
      4                  1548938093827  
...                                ...  
test  16914              1555517283557  
      16915              1556231324046  
      16916              1555791340920  
      16917              1555619158162  
      16918              1557406106800  

[836435 rows x 5 columns]

In [5]:
search.columns

Index(['session_id_hash', 'query_vector', 'clicked_skus_hash',
       'product_skus_hash', 'server_timestamp_epoch_ms'],
      dtype='object')

In [6]:
info.idx2act

{0: 'detail', 1: 'pageview', 2: 'add', 3: 'purchase', 4: 'remove'}

In [7]:
browsing = browsing[browsing['product_action'] != 4].copy()

In [8]:
url2item = browsing.dropna().drop_duplicates(subset=['hashed_url', 'product_sku_hash']).groupby('hashed_url')['product_sku_hash'].agg(list)
url2item = url2item.reset_index()
url2item['product_sku_hash'] = url2item['product_sku_hash'].apply(lambda x: x[0])
url2item = np.vstack(url2item.values).T
url2item = url2item.astype(int).tolist()
url2item = dict(zip(*url2item))
browsing['product_sku_hash'] = browsing['hashed_url'].map(url2item.get)

In [15]:
browsing.drop_duplicates(subset=['session_id_hash', 'hashed_url'], keep='last', inplace=True)
browsing = browsing.sort_values(by=['session_id_hash', 'server_timestamp_epoch_ms'])
browsing['hashed_url'] = browsing['hashed_url'].astype('int')
# browsing['product_sku_hash'] = browsing['product_sku_hash'].astype('int')

In [16]:
browsing = browsing[['session_id_hash', 'product_sku_hash', 'hashed_url', 'server_timestamp_epoch_ms']]
browsing.columns = ['sess_id', 'sku_id', 'url_id', 'timestamp']

In [17]:
browsing_train = browsing.loc['train', :].copy()
browsing_test = browsing.loc['test', :].copy()

In [18]:
train = browsing_train.copy()
test = browsing_test.copy()

In [20]:
train['url_count'] = train.groupby('sess_id')['url_id'].transform('count')
train = train[train['url_count'] >=2]
train.drop(columns=['url_count'], inplace=True)

In [21]:
train.shape, test.shape

((14928146, 4), (424540, 4))

In [22]:
train

,sess_id,sku_id,url_id,timestamp
16758591,0,NaN,2187,1550884057392
16758599,0,42112.0,5870,1550885056208
16758600,0,NaN,3,1550885061564
16758612,0,30494.0,1,1550885205035
16758613,0,31231.0,0,1550885210881
...,...,...,...,...
36079285,4934698,NaN,5,1550870912491
36079286,4934698,NaN,179,1550870916250
36079287,4934698,NaN,96,1550870946172
36079288,4934698,NaN,16,1550870968864


In [24]:
test_pad = test.groupby('sess_id').tail(1).copy()
test_pad['timestamp'] += 1
test_dev = pd.concat([test, test_pad])

In [25]:
test_dev = test_dev.sort_values(by=['sess_id', 'timestamp'])
train = train.sort_values(by=['sess_id', 'timestamp'])

In [26]:
train.fillna(-1, inplace=True)
test_dev.fillna(-1, inplace=True)

In [27]:
train['sku_id'] = train['sku_id'].astype('int')
test_dev['sku_id'] = test_dev['sku_id'].astype('int')

In [28]:
train_set = []
valid_set = []
test_set = []

In [29]:
seq_len = 3
for sess_id, data_id in tqdm(train.groupby('sess_id')[['sku_id', 'url_id']]):
    url_ids = data_id['url_id'].tolist()
    sku_ids = data_id['sku_id'].tolist()
    his_mask = data_id['sku_id'].tolist()
    for idx in range(1, len(url_ids)-1):
        if idx < seq_len:
            seq = url_ids[:idx] + [-1] * (seq_len - idx)
        else:
            seq = url_ids[idx - seq_len:idx]
        target = sku_ids[idx]
        if target < 0:
            continue
        train_set.append((sess_id, seq, his_mask[:idx], int(target)))
        
    idx = len(url_ids)-1
    if idx < seq_len:
        seq = url_ids[:idx] + [-1] * (seq_len - idx)
    else:
        seq = url_ids[idx - seq_len:idx]
    target = sku_ids[idx]
    if target < 0:
        continue
    valid_set.append((sess_id, seq, his_mask[:idx], int(target)))  

100%|██████████| 2573090/2573090 [04:32<00:00, 9443.50it/s] 


In [30]:
len(train_set), len(valid_set)

(4337037, 1197025)

In [31]:
test_dev

,sess_id,sku_id,url_id,timestamp
0,4934699,391,19977,1557432435044
0,4934699,391,19977,1557432435045
1,4934700,13483,6506,1555443439867
1,4934700,13483,6506,1555443439868
2,4934701,-1,428,1556729588695
...,...,...,...,...
559513,5076752,13492,24609,1557406442987
559514,5076753,57288,176,1557337655966
559514,5076753,57288,176,1557337655967
559515,5076754,36654,4252,1556234499481


In [32]:
for sess_id, data_id in tqdm(test_dev.groupby('sess_id')[['sku_id', 'url_id']]):
    url_ids = data_id['url_id'].tolist()
    sku_ids = data_id['sku_id'].tolist()
    his_mask = data_id['sku_id'].tolist()
    for idx in range(1, len(url_ids)-1):
        if idx < seq_len:
            seq = url_ids[:idx] + [-1] * (seq_len - idx)
        else:
            seq = url_ids[idx - seq_len:idx]
        target = sku_ids[idx]
        if target < 0:
            continue
        train_set.append((sess_id, seq, his_mask[:idx], int(target)))
        
    idx = len(url_ids)-1
    if idx < seq_len:
        seq = url_ids[:idx] + [-1] * (seq_len - idx)
    else:
        seq = url_ids[idx - seq_len:idx]
    target = sku_ids[idx]
    if target < 0:
        continue
    test_set.append((sess_id, seq, his_mask[:idx], int(target)))

100%|██████████| 142042/142042 [00:13<00:00, 10914.81it/s]


In [33]:
train_set += valid_set[:1100000]
valid_set = valid_set[1100000:]

In [34]:
len(train_set), len(valid_set), len(test_set)

(5560178, 97025, 77196)

In [35]:
train

,sess_id,sku_id,url_id,timestamp
16758591,0,-1,2187,1550884057392
16758599,0,42112,5870,1550885056208
16758600,0,-1,3,1550885061564
16758612,0,30494,1,1550885205035
16758613,0,31231,0,1550885210881
...,...,...,...,...
36079285,4934698,-1,5,1550870912491
36079286,4934698,-1,179,1550870916250
36079287,4934698,-1,96,1550870946172
36079288,4934698,-1,16,1550870968864


In [36]:
for name, data in zip(['url_train', 'url_valid', 'url_test'], [train_set, valid_set, test_set]):
    pathname = f'../dataset/prepared/{name}.csv'
    with open(pathname, 'w') as f:
        for user_id, seq, mask, target in data:
            f.write(','.join([str(user_id), '|'.join(map(str, seq)), '|'.join(map(str, mask)), str(target)]))
            f.write('\n')

In [39]:
np.save('../dataset/prepared/url_test_items.npy', test_items)